In [ ]:
import logging
from datetime import datetime

from src.surrealengine import (
    Document, StringField, DateTimeField,
    create_connection
)
from src.surrealengine.signals import pre_save, post_save, pre_init, post_init, handler

In [ ]:
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Define a document class
class User(Document):
    name = StringField(required=True)
    email = StringField()
    created_at = DateTimeField()
    updated_at = DateTimeField()

    # Define signal handlers as class methods
    @classmethod
    def pre_save_handler(cls, sender, document, **kwargs):
        logger.info(f"Pre-save: {document.name}")
        # Update the updated_at field
        document.updated_at = datetime.utcnow()

    @classmethod
    def post_save_handler(cls, sender, document, **kwargs):
        created = kwargs.get('created', False)
        action = "Created" if created else "Updated"
        logger.info(f"Post-save: {action} {document.name}")

# Define a standalone signal handler
def pre_init_handler(sender, document, values, **kwargs):
    logger.info(f"Pre-init: {sender.__name__} with values: {values}")
    # You can modify the values here if needed
    if 'created_at' not in values:
        values['created_at'] = datetime.utcnow()

# Define a handler using the decorator
@handler(post_init)
def post_init_handler(sender, document, **kwargs):
    logger.info(f"Post-init: {document.__class__.__name__} initialized")
    return document

# Connect the signal handlers
pre_init.connect(pre_init_handler, sender=User)
pre_save.connect(User.pre_save_handler, sender=User)
post_save.connect(User.post_save_handler, sender=User)

In [ ]:
connection = create_connection(
        url="ws://db:8000/rpc",
        namespace="test_ns",
        database="test_db",
        username="root",
        password="root",
        make_default=True,
    )
await connection.connect()

In [ ]:
# Create a new user
user = User(name="John Doe", email="john@example.com")
logger.info(f"User created: {user.name}, {user.email}, created_at: {user.created_at}")

# Save the user
await user.save()
logger.info(f"User saved with ID: {user.id}")

In [ ]:
user.name = "Jane Doe"
await user.save()
logger.info(f"User updated: {user.name}, updated_at: {user.updated_at}")

In [ ]:
# Delete the user
await user.delete()
logger.info("User deleted")

In [ ]:
# Bulk create users
users = [
    User(name=f"User {i}", email=f"user{i}@example.com")
    for i in range(5)
]
created_users = await User.bulk_create(users)
logger.info(f"Created {len(created_users)} users in bulk")

In [ ]:
for u in created_users:
    await u.delete()